---
title: "Trendsetter Challenge: Analyzing Trends & Social Influence in Spatial-Temporal Data"
description: "Data analyst take-home exam, focusing data preprocessing and spatial analysis to filter through vast datasets, pinpointing the most influential users and calculating their spatial coverage."
author:
  - name: Piero Trujillo
    url: https://suppiero.github.io 

date: 2023-10-18
categories: [PYTHON, DATA WRANGLING & CLEANING, DATA STRUCTURES, DATA ANALYST TAKE HOME EXAM]
citation:
  url: https://suppiero.github.io/projects/trendsetter_challenge/
#bibliography: references.bib 
format:
  #html:
  #  code-fold: true #false
  #  code-summary: "code dropdown"
image: "airport.jpg"
draft: false # "true" will mean this is a draft post so it wont show up on my site
---

### Introduction

The Trendsetter Challenge was a data analytics assessment I completed for the leading insurance data company in my area as part of their technical interview process. The dataset contains around 6.5 million observations and my final solution took around 2 minutes to run (on my nine-year-old laptop 😅). 

The definitions and deliverables for this challenge are as follows:

#### Definitions
* Assume a “trendsetter” is someone who, in this dataset, visits some of the same locations as another person, before the other person goes to that location.
* A “trendsetter score” for a person is the number of distinct locations and people that have visited a location after the "trendsetter".
* A "radius of influence" for a trendsetter is an approximate measurement of the radius of the circle that bounds the places that person has checked in. Assume the earth is flat.

#### Deliverables
* Two tables containing ten rows including the following information:
    * user_id of the trendsetter
    * Trendsetter score
    * Radius of influence
* First table displays trendsetters ranked by score, highest to lowest.
* Second table displays trendsetters ranked by radius of influence, highest to lowest.

### Understanding our Dataset

The dataset utilized in this project provides spatial and temporal information on various people and the locations they visited.

**Data**

| column | description |
| --- | --- |
| user_id | Trendsetter ID |
| check_in_time | Time a Trendsetter arrived at a location ID |
| latitude | location's latitude coordinates |
| longitude | location's longitude coordinates |
| location_id | ID for a given location |


Feel free to look through my solution and access the data on my [GitHub repository](https://github.com/suppiero/trendsetter_challenge). Now, let’s dive into the Trendsetter Challenge!

### Setup and Preprocessing

In [1]:
import pandas as pd
# read in txt file, separated by space to create tabular dataframe 
total_checkins = pd.read_csv("/Users/piero/Downloads/trendsetter_challenge_project/loc-gowalla_totalCheckins.txt", sep="	", header = None) 
edges = pd.read_csv("/Users/piero/Downloads/trendsetter_challenge_project/loc-gowalla_edges.txt", sep="	", header = None) 

In [2]:
# Adding column headings 
total_checkins.columns = ['user_id', 'check_in_time', 'latitude', 'longitude', 'location_id'] 
#edges.columns = ['Friend A', 'Friend B'] # column names based on the research paper

# Verify output is in tabular format
print(total_checkins.head(5))

   user_id         check_in_time   latitude  longitude  location_id
0        0  2010-10-19T23:55:27Z  30.235909 -97.795140        22847
1        0  2010-10-18T22:17:43Z  30.269103 -97.749395       420315
2        0  2010-10-17T23:42:03Z  30.255731 -97.763386       316637
3        0  2010-10-17T19:26:05Z  30.263418 -97.757597        16516
4        0  2010-10-16T18:50:42Z  30.274292 -97.740523      5535878


In [3]:
# Check Data Types of Columns
total_checkins.dtypes

# Convert Check-In-Time from an Object to Datetime
total_checkins['check_in_time'] = pd.to_datetime(total_checkins['check_in_time'])

# Make Sure Check-In-Time is of Type Datetime
print(total_checkins.dtypes)
print(total_checkins.head(5))

user_id                        int64
check_in_time    datetime64[ns, UTC]
latitude                     float64
longitude                    float64
location_id                    int64
dtype: object
   user_id             check_in_time   latitude  longitude  location_id
0        0 2010-10-19 23:55:27+00:00  30.235909 -97.795140        22847
1        0 2010-10-18 22:17:43+00:00  30.269103 -97.749395       420315
2        0 2010-10-17 23:42:03+00:00  30.255731 -97.763386       316637
3        0 2010-10-17 19:26:05+00:00  30.263418 -97.757597        16516
4        0 2010-10-16 18:50:42+00:00  30.274292 -97.740523      5535878


In [4]:
# First, sort to find the earliest user check-ins for each location
# Go by location ID first and then sort by date
trendsetters = total_checkins.sort_values(['location_id', 'check_in_time'],ascending=True)

# Create a table to get each unique location ID
location_table = total_checkins['location_id'].unique()
# Create a table to get each unique user ID
user_table = total_checkins['user_id'].unique()

# Second, group each location by user so we have each user's earliest arrival
#trendsetters = trendsetters.groupby(["location_id", "check_in_time"])#.head(10)
print(trendsetters.head(100))

         user_id             check_in_time   latitude  longitude  location_id
6282          24 2009-02-05 06:27:43+00:00  39.052318 -94.607499         8904
101571       343 2009-03-08 02:09:11+00:00  39.052318 -94.607499         8904
101540       343 2009-05-28 23:40:31+00:00  39.052318 -94.607499         8904
76716        256 2009-05-30 03:33:23+00:00  39.052318 -94.607499         8904
101531       343 2009-06-20 01:40:01+00:00  39.052318 -94.607499         8904
...          ...                       ...        ...        ...          ...
153650       488 2010-01-29 19:09:15+00:00  39.052824 -94.590311         8938
153645       488 2010-02-03 19:02:39+00:00  39.052824 -94.590311         8938
153644       488 2010-02-04 18:52:36+00:00  39.052824 -94.590311         8938
2910820    36254 2010-02-04 22:37:43+00:00  39.052824 -94.590311         8938
925993      4207 2010-02-06 18:40:45+00:00  39.052824 -94.590311         8938

[100 rows x 5 columns]


In [5]:
# Create groups for each location where we count every user_id that's been there (keep lat & lon columns)
count_series = trendsetters.groupby(['location_id', 'user_id', 'latitude', 'longitude'], sort=False).size()
print(count_series.head(100))

location_id  user_id  latitude   longitude  
8904         24       39.052318  -94.607499     1
             343      39.052318  -94.607499     4
             256      39.052318  -94.607499     3
             576      39.052318  -94.607499     1
             310      39.052318  -94.607499     1
                                               ..
8947         8022     37.331880  -122.029631    4
             125329   37.331880  -122.029631    9
             25326    37.331880  -122.029631    1
             127250   37.331880  -122.029631    2
             157368   37.331880  -122.029631    1
Length: 100, dtype: int64


In [6]:
# Add visit frequency of each user as a column to series
new_df = count_series.to_frame(name = 'visit_frequency').reset_index()

# Find distinct amount of visitors per location
unique_location_visitors = trendsetters.groupby('location_id')['user_id'].nunique()

# Add total visitors column
# Currently have a series so we convert it to a dataframe
unique_location_visitors = pd.DataFrame(unique_location_visitors).reset_index()
unique_location_visitors.columns = ['location_id', 'total_visitors']

# Merge on location_id to grab the total visitors who attended a location
merged_df = pd.merge(left=new_df, right=unique_location_visitors, how='left', left_on='location_id', right_on='location_id')
print(merged_df.head(100))

# Create empty score column
merged_df["trendsetter_score"] = None

    location_id  user_id   latitude   longitude  visit_frequency  \
0          8904       24  39.052318  -94.607499                1   
1          8904      343  39.052318  -94.607499                4   
2          8904      256  39.052318  -94.607499                3   
3          8904      576  39.052318  -94.607499                1   
4          8904      310  39.052318  -94.607499                1   
..          ...      ...        ...         ...              ...   
95         8947     8022  37.331880 -122.029631                4   
96         8947   125329  37.331880 -122.029631                9   
97         8947    25326  37.331880 -122.029631                1   
98         8947   127250  37.331880 -122.029631                2   
99         8947   157368  37.331880 -122.029631                1   

    total_visitors  
0                7  
1                7  
2                7  
3                7  
4                7  
..             ...  
95             268  
96             

# Completing the Trendsetter Challenge With a Subset of Data
This was necessary in order to practice and make sure the logic of my solution is correct. Feel free to skip to the implementation with the full dataset below!

### Creating a Subset of Data 

In [7]:
# Create a subset of data to practice our loops and logic on
subset_data = merged_df.head(200)
print(subset_data.head(20))

    location_id  user_id   latitude  longitude  visit_frequency  \
0          8904       24  39.052318 -94.607499                1   
1          8904      343  39.052318 -94.607499                4   
2          8904      256  39.052318 -94.607499                3   
3          8904      576  39.052318 -94.607499                1   
4          8904      310  39.052318 -94.607499                1   
5          8904      392  39.052318 -94.607499                1   
6          8904     5164  39.052318 -94.607499                1   
7          8932    32391  32.927662 -97.254356                1   
8          8932      350  32.927662 -97.254356                1   
9          8932     6046  32.927662 -97.254356                1   
10         8932     4648  32.927662 -97.254356                2   
11         8932   105294  32.927662 -97.254356                1   
12         8932    44458  32.927662 -97.254356                2   
13         8932     4678  32.927662 -97.254356                

### For Loop To Rank Users By Position At Each Location (Multiple People Can Be Trendsetters)

In [9]:
import pandas as pd
from collections import defaultdict

# Initialize empty defaultdict to store user positions 
# used defaultdict instead of dict as a safety net against missing [user,location] pairs
scores = defaultdict(list)

# Initialize current_location and current_users
current_location = -1
current_users = []  # List to store users who have already visited the current location

subset_data['position'] = 0  # Create a new position column where everyone starts at 0

position_count = 1  # Counter used to keep track of ranked position at each location

# Create an empty list to store every user's attributes inside a dictionary 
# Used a list to store multiple instances of the same user (dictionary won't allow duplicates)
data = []

for index, row in subset_data.iterrows():
    # Elements at the current row of the dataframe
    location_id = row['location_id']
    user_id = row['user_id']
    position = row['position']
    latitude = row['latitude']
    longitude = row['longitude']

    # New location 
    if location_id != current_location:
        current_location = location_id
        current_users = []  # Reset the list of users for the new location
        position_count = 1  # Reset the count

    # Create a dictionary for the current user_id
    item = {
        'user_id': user_id,
        'position': position_count,
        'location_id': location_id,
        'latitude': latitude,
        'longitude': longitude
    }

     # Add dictionary of current user_id to list
    data.append(item)

    # First time user is at this location
    if user_id not in current_users:
        current_users.append(user_id) # add user_id to list of users who have been here

        # We have never recorded this user at any location
        if user_id not in scores:
            position_count += 1

            # Store a user's attributes in the dictionary
            scores[user_id].append({
                'position': position_count,
                'location_id': current_location,
                'latitude': latitude,
                'longitude': longitude
            }) 

        # Returning user but at a new location  
        else:
            position_count += 1

            # Store a user's attributes in the dictionary
            scores[user_id].append({
                'position': position_count,
                'location_id': current_location,
                'latitude': latitude,
                'longitude': longitude
            })

# Create DataFrame from list of every user's dictionary data 
position_df = pd.DataFrame(data)

# Print the first three rows of the DataFrame
print(position_df.head(3))

   user_id  position  location_id   latitude  longitude
0       24         1         8904  39.052318 -94.607499
1      343         2         8904  39.052318 -94.607499
2      256         3         8904  39.052318 -94.607499


/var/folders/th/h7c9tz61505fhg7ds00qjhlm0000gn/T/ipykernel_831/4145353881.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['position'] = 0  # Create a new position column where everyone starts at 0


### Finding the Distinct Amount of Users at Every Location

In [10]:
# Find distinct amount of visitors per location
unique_location_visitors = position_df.groupby('location_id')['user_id'].nunique()

# Add total visitors column
# Currently have a series so convert to DataFrame
unique_location_visitors = pd.DataFrame(unique_location_visitors).reset_index()
unique_location_visitors.columns = ['location_id', 'total_visitors']

# Merge on location_id to grab the total visitors who attended a location
ranked_data = pd.merge(left=position_df, right=unique_location_visitors, how='left', left_on='location_id', right_on='location_id')
print(ranked_data.head(100))

    user_id  position  location_id   latitude   longitude  total_visitors
0        24         1         8904  39.052318  -94.607499               7
1       343         2         8904  39.052318  -94.607499               7
2       256         3         8904  39.052318  -94.607499               7
3       576         4         8904  39.052318  -94.607499               7
4       310         5         8904  39.052318  -94.607499               7
..      ...       ...          ...        ...         ...             ...
95     8022        41         8947  37.331880 -122.029631             145
96   125329        42         8947  37.331880 -122.029631             145
97    25326        43         8947  37.331880 -122.029631             145
98   127250        44         8947  37.331880 -122.029631             145
99   157368        45         8947  37.331880 -122.029631             145

[100 rows x 6 columns]


### Calculating The Trendsetter Score

In [11]:
# Formula: total_visitors - position = trendsetter_score
ranked_data['trendsetter_score'] = ranked_data['total_visitors'] - ranked_data['position']
ranked_data

# Sort dataframe by trendsetter_score from highest to lowest
ranked_data = ranked_data.sort_values('trendsetter_score', ascending=False)

ranked_data

,user_id,position,location_id,latitude,longitude,total_visitors,trendsetter_score
55,130,1,8947,37.331880,-122.029631,145,144
56,254,2,8947,37.331880,-122.029631,145,143
57,24,3,8947,37.331880,-122.029631,145,142
58,602,4,8947,37.331880,-122.029631,145,141
59,494,5,8947,37.331880,-122.029631,145,140
...,...,...,...,...,...,...,...
6,5164,7,8904,39.052318,-94.607499,7,0
54,447,28,8938,39.052824,-94.590311,28,0
19,31698,13,8932,32.927662,-97.254356,13,0
26,9767,7,8936,39.053318,-94.591995,7,0


### Total Trendsetter Score For Every User

In [12]:
# To find the total trendsetter score, add the scores for rows where the user_id is the same
total_scores = ranked_data.groupby('user_id')['trendsetter_score'].sum().reset_index()

# Sort users from highest to lowest score
total_scores = total_scores.sort_values(by='trendsetter_score', ascending=False)

total_scores

,user_id,trendsetter_score
1,24,148
8,130,144
14,254,143
43,602,141
34,494,140
...,...,...
152,42960,1
29,447,0
105,9767,0
142,31698,0


### Haversine Distance Formula

In [186]:
import math

# Calculate a user's largest radius
def get_radius_from_lat_lon_in_km(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    d_lat = deg2rad(lat2 - lat1)  # deg2rad function defined below
    d_lon = deg2rad(lon2 - lon1)
    a = (
        math.sin(d_lat/2) * math.sin(d_lat/2) +
        math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) *
        math.sin(d_lon/2) * math.sin(d_lon/2)
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in kilometers
    radius = d / 2 # radius = distance / 2
    return radius

# convert coordinates to radians
def deg2rad(deg):
    return deg * (math.pi / 180)

# Example usage for user_id 24
lat1, lon1 = 37.331880,	-122.029631
lat2, lon2 = 39.052318,	-94.607499

radius = get_radius_from_lat_lon_in_km(lat1, lon1, lat2, lon2)
print(f"The radius between the two points is {radius} kilometers.")
# checked answer with this calculator on google, it works! Just divide answer by 2: (https://calculator.academy/haversine-distance-calculator/)

The radius between the two points is 1197.5154698023875 kilometers.


### Preprocessing to Find the Largest Radius of All Users

In [14]:
# Merge datasets to get trendsetter score column 
complete_df = ranked_data.merge(total_scores, on='user_id', how='left')

complete_df.drop(columns=['trendsetter_score_x'], axis=1, inplace=True)

complete_df = complete_df.rename(columns={'trendsetter_score_y':'trendsetter_score'})

complete_df

,user_id,position,location_id,latitude,longitude,total_visitors,trendsetter_score
0,130,1,8947,37.331880,-122.029631,145,144
1,254,2,8947,37.331880,-122.029631,145,143
2,24,3,8947,37.331880,-122.029631,145,148
3,602,4,8947,37.331880,-122.029631,145,141
4,494,5,8947,37.331880,-122.029631,145,140
...,...,...,...,...,...,...,...
195,5164,7,8904,39.052318,-94.607499,7,19
196,447,28,8938,39.052824,-94.590311,28,0
197,31698,13,8932,32.927662,-97.254356,13,0
198,9767,7,8936,39.053318,-94.591995,7,0


The loop will only work with users who have visited multiple locations because the distance function needs multiple coordinates to calculate distance. Therefore, remove all users with one location visited. Also, anyone with a trendsetter score of 0 is not a trendsetter.

In [56]:
# Remove rows from dataframe where user_id only appears once
filtered_df = complete_df[complete_df.groupby('user_id').user_id.transform('count') > 1]

# Remove users with a trendsetter score of 0
filtered_df = filtered_df[filtered_df['trendsetter_score'] > 0]

### Calculate the Largest Distance a User Has Traveled Based on All of the Coordinate Pairs They Have Been To (Brute Force Approach)

In [61]:
import numpy as np
from itertools import combinations

# Initialize dictionary to store the largest distance for each user
largest_distances = {}

# Group data by distinct user_id to iterate through each user
distance_data = filtered_df.groupby('user_id')

# Iterate through each user and calculate their largest distance (user_group = all the rows belonging to current user)
for user, user_group in distance_data:
    distances = [] # list to store all of a user's distances

    # Create combinations of every user's latitude & longitude pairs at each user index
    # Then, calculate the distance between all pairs of coordinates for a user
    for pair in combinations(user_group.index, 2):
        lat1, lon1 = user_group.loc[pair[0], ['latitude', 'longitude']]
        lat2, lon2 = user_group.loc[pair[1], ['latitude', 'longitude']]
        distances.append(get_radius_from_lat_lon_in_km(lat1, lon1, lat2, lon2)) # add each radius to list

    # Find the largest radius for this user
    largest_distance = max(distances)
    largest_distances[user] = largest_distance  # keep track of the largest radius for each user in dictionary

# EDA: Finding user with the largest distance to verify table is correct
max_user = max(largest_distances, key=largest_distances.get)

print("User with the largest radius:", max_user)
print("Largest radius:", largest_distances[max_user])

User with the largest radius: 392
Largest radius: 1198.2536467390876


### Create Final Tables

In [49]:
# Convert the dictionary to a DataFrame
largest_radius_df = pd.DataFrame(list(largest_distances.items()), columns=['user_id', 'largest_radius'])

# Add column displaying the largest distance for each user
final_df = largest_radius_df.merge(filtered_df, on='user_id', how='left')

# Remove duplicate user_id rows
final_df = final_df.drop_duplicates(subset=['user_id'])

# Drop Unnecessary Columns
final_df.drop(columns=['position', 'longitude', 'latitude', 'location_id', 'total_visitors'], axis=1, inplace=True)

# Sort by trendsetter_score & radius to create final tables
final_score_df = final_df.sort_values(by=['trendsetter_score'], ascending=False)
final_radius_df = final_df.sort_values(by=['largest_radius'], ascending=False)

## Deliverables

### Trendsetters Ranked By Score (Highest to Lowest)

In [50]:
final_score_df

,user_id,largest_radius,trendsetter_score
0,24,1197.515486,148
4,392,1198.253647,66
10,576,0.742610,28
12,4893,0.077745,24
2,256,0.742610,20
14,5164,0.742610,19
8,542,0.077745,7


### Trendsetters Ranked By Radius of Influence (Highest to Lowest)

In [51]:
final_radius_df

,user_id,largest_radius,trendsetter_score
4,392,1198.253647,66
0,24,1197.515486,148
2,256,0.742610,20
10,576,0.742610,28
14,5164,0.742610,19
8,542,0.077745,7
12,4893,0.077745,24


# Completing The Trendsetter Challenge With A Complete Dataset

### For Loop To Rank Users By Position At Each Location (Multiple People Can Be Trendsetters)

In [7]:
import pandas as pd
from collections import defaultdict

# Initialize empty defaultdict to store user positions 
# used defaultdict instead of dict as a safety net against missing [user,location] pairs
scores = defaultdict(list)

# Initialize current_location and current_users
current_location = -1
current_users = []  # List to store users who have already visited the current location

merged_df['position'] = 0  # Create a new position column where everyone starts at 0

position_count = 1  # Counter used to keep track of ranked position at each location

# Create an empty list to store every user's attributes inside a dictionary 
# Used a list to store multiple instances of the same user (dictionary won't allow duplicates)
data = []

for index, row in merged_df.iterrows():
    # Elements at the current row of the dataframe
    location_id = row['location_id']
    user_id = row['user_id']
    position = row['position']
    latitude = row['latitude']
    longitude = row['longitude']

    # New location 
    if location_id != current_location:
        current_location = location_id
        current_users = []  # Reset the list of users for the new location
        position_count = 1  # Reset the count

    # Create a dictionary for the current user_id
    item = {
        'user_id': user_id,
        'position': position_count,
        'location_id': location_id,
        'latitude': latitude,
        'longitude': longitude
    }

     # Add dictionary of current user_id to list
    data.append(item)

    # First time user is at this location
    if user_id not in current_users:
        current_users.append(user_id) # add user_id to list of users who have been here

        # We have never recorded this user at any location
        if user_id not in scores:
            position_count += 1

            # Store a user's attributes in the dictionary
            scores[user_id].append({
                'position': position_count,
                'location_id': current_location,
                'latitude': latitude,
                'longitude': longitude
            }) 

        # Returning user but at a new location  
        else:
            position_count += 1

            # Store a user's attributes in the dictionary
            scores[user_id].append({
                'position': position_count,
                'location_id': current_location,
                'latitude': latitude,
                'longitude': longitude
            })

# Create DataFrame from list of every user's dictionary data 
position_df = pd.DataFrame(data)

# Print the first three rows of the DataFrame
print(position_df.head(3))

   user_id  position  location_id   latitude  longitude
0       24         1         8904  39.052318 -94.607499
1      343         2         8904  39.052318 -94.607499
2      256         3         8904  39.052318 -94.607499


### Finding the Distinct Amount of Users at Every Location

In [8]:
# Find distinct amount of visitors per location
unique_location_visitors = position_df.groupby('location_id')['user_id'].nunique()

# Add total visitors column
# Currently have a series so convert to DataFrame
unique_location_visitors = pd.DataFrame(unique_location_visitors).reset_index()
unique_location_visitors.columns = ['location_id', 'total_visitors']

# Merge on location_id to grab the total visitors who attended a location
ranked_data = pd.merge(left=position_df, right=unique_location_visitors, how='left', left_on='location_id', right_on='location_id')
print(ranked_data.head(100))

    user_id  position  location_id   latitude   longitude  total_visitors
0        24         1         8904  39.052318  -94.607499               7
1       343         2         8904  39.052318  -94.607499               7
2       256         3         8904  39.052318  -94.607499               7
3       576         4         8904  39.052318  -94.607499               7
4       310         5         8904  39.052318  -94.607499               7
..      ...       ...          ...        ...         ...             ...
95     8022        41         8947  37.331880 -122.029631             268
96   125329        42         8947  37.331880 -122.029631             268
97    25326        43         8947  37.331880 -122.029631             268
98   127250        44         8947  37.331880 -122.029631             268
99   157368        45         8947  37.331880 -122.029631             268

[100 rows x 6 columns]


### Calculating the Trendsetter Score

In [9]:
# Formula: total_visitors - position = trendsetter_score
ranked_data['trendsetter_score'] = ranked_data['total_visitors'] - ranked_data['position']
ranked_data

# Sort dataframe by trendsetter_score from highest to lowest
ranked_data = ranked_data.sort_values('trendsetter_score', ascending=False)

ranked_data

,user_id,position,location_id,latitude,longitude,total_visitors,trendsetter_score
219955,130,1,19542,37.616356,-122.386150,2631,2630
219956,5831,2,19542,37.616356,-122.386150,2631,2629
219957,5000,3,19542,37.616356,-122.386150,2631,2628
219958,427,4,19542,37.616356,-122.386150,2631,2627
219959,5118,5,19542,37.616356,-122.386150,2631,2626
...,...,...,...,...,...,...,...
2793405,183112,1,875737,51.815994,-0.354031,1,0
2793404,2676,1,875735,41.382403,2.174088,1,0
2793403,130675,1,875733,60.194537,21.910596,1,0
2793402,20329,2,875732,51.179708,8.487231,2,0


### Total Trendsetter Score For Every User

In [10]:
# To find the total trendsetter score, add the scores for rows where the user_id is the same
total_scores = ranked_data.groupby('user_id')['trendsetter_score'].sum().reset_index()

# Sort users from highest to lowest score
total_scores = total_scores.sort_values(by='trendsetter_score', ascending=False)

total_scores

,user_id,trendsetter_score
32111,48860,60714
20,22,58371
243,267,49310
366,401,47750
536,588,45694
...,...,...
78579,133819,0
78582,133826,0
78584,133828,0
78592,133849,0


### Preprocessing to Find the Largest Radius of All Users

In [18]:
# Merge datasets to get trendsetter score column 
complete_df = ranked_data.merge(total_scores, on='user_id', how='left')

# Drop Unnecessary Columns
complete_df.drop(columns=['trendsetter_score_x'], axis=1, inplace=True)

# Rename 'trendsetter_score' column
complete_df = complete_df.rename(columns={'trendsetter_score_y':'trendsetter_score'})

complete_df

,user_id,position,location_id,latitude,longitude,total_visitors,trendsetter_score
0,130,1,19542,37.616356,-122.386150,2631,40486
1,5831,2,19542,37.616356,-122.386150,2631,18826
2,5000,3,19542,37.616356,-122.386150,2631,5264
3,427,4,19542,37.616356,-122.386150,2631,4168
4,5118,5,19542,37.616356,-122.386150,2631,11013
...,...,...,...,...,...,...,...
3981329,183112,1,875737,51.815994,-0.354031,1,296
3981330,2676,1,875735,41.382403,2.174088,1,4483
3981331,130675,1,875733,60.194537,21.910596,1,7551
3981332,20329,2,875732,51.179708,8.487231,2,42


The loop will only work with users who have visited multiple locations because the distance function needs multiple coordinates to calculate distance. Therefore, remove all users with one location visited. Also, anyone with a trendsetter score of 0 is not a trendsetter.

In [101]:
# Remove rows from dataframe where user_id only appears once
filtered_df = complete_df[complete_df.groupby('user_id').user_id.transform('count') > 1]

# Remove users with a trendsetter score of 0
filtered_df = filtered_df[filtered_df['trendsetter_score'] > 0]

In [102]:
filtered_df

,user_id,position,location_id,latitude,longitude,total_visitors,trendsetter_score
0,130,1,19542,37.616356,-122.386150,2631,40486
1,5831,2,19542,37.616356,-122.386150,2631,18826
2,5000,3,19542,37.616356,-122.386150,2631,5264
3,427,4,19542,37.616356,-122.386150,2631,4168
4,5118,5,19542,37.616356,-122.386150,2631,11013
...,...,...,...,...,...,...,...
3981329,183112,1,875737,51.815994,-0.354031,1,296
3981330,2676,1,875735,41.382403,2.174088,1,4483
3981331,130675,1,875733,60.194537,21.910596,1,7551
3981332,20329,2,875732,51.179708,8.487231,2,42


### Preprocessing for Convex Hull

In [103]:
# Sort dataframe by user_id so all of their coordinates are together
filtered_df = filtered_df.sort_values(by='user_id')

# Drop unnecesary columns
filtered_df = filtered_df.drop(columns=['position', 'total_visitors'])

filtered_df

,user_id,location_id,latitude,longitude,trendsetter_score
488829,0,19199,37.785436,-122.403949,9506
3709485,0,87914,40.731724,-74.003338,9506
181109,0,9382,30.263886,-97.737980,9506
730242,0,16002,37.788192,-122.401214,9506
185715,0,15245,30.264298,-97.732980,9506
...,...,...,...,...,...
2671648,196578,616571,51.745659,-0.481369,17
3654637,196578,965051,51.745971,-0.485233,17
3767879,196578,1072999,51.749802,-0.501034,17
3252652,196578,635712,51.749478,-0.500603,17


### Filter Out Coordinates That Are Close Together Using KDTree

In [108]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# Set radius for filtering (input array units, really close coordinates yield ~0.1 distance)
radius = 0.1  

# Store every user's data in a list
closest_locations_data = []

# Iterate through each user and calculate the distance from each of their coordinate pairs
for user, user_data in filtered_df.groupby('user_id'):
    coordinates = user_data[['latitude', 'longitude']].to_numpy() # convert all user coordinates to array
    tree = cKDTree(coordinates)

    # Query KDTree to find nearby coordinates
    distances, indexes = tree.query(coordinates, k=2) # find 2nd closest distance (1st will be the original point)

    # Store a new dataframe for each user to avoid overwriting
    current_user_data = user_data.copy() 

    # Record the closest location to closest distance
    current_user_data['closest_location'] = current_user_data['location_id'].iloc[indexes[:, 1]].values
    current_user_data['closest_dist'] = distances[:, 1]

    # Remove duplicate nearest distances rows
    current_user_data.drop_duplicates(['closest_dist'], keep='first', inplace=True)
    
    # Append user's dataframe to the list
    closest_locations_data.append(current_user_data)

In [109]:
# Concatenate list of user DataFrames
closest_locations_df = pd.concat(closest_locations_data, ignore_index=True)

closest_locations_df
# It worked! I cleaned up 1186811 rows! (3953500 - 2766689)

         user_id  location_id   latitude   longitude  trendsetter_score  \
0              0        19199  37.785436 -122.403949               9506   
1              0        87914  40.731724  -74.003338               9506   
2              0         9382  30.263886  -97.737980               9506   
3              0        16002  37.788192 -122.401214               9506   
4              0        15245  30.264298  -97.732980               9506   
...          ...          ...        ...         ...                ...   
2766684   196578       965121  51.746492   -0.490780                 17   
2766685   196578       697962  51.749451   -0.500988                 17   
2766686   196578      1151847  51.749706   -0.500992                 17   
2766687   196578       616571  51.745659   -0.481369                 17   
2766688   196578       635712  51.749478   -0.500603                 17   

         closest_location  closest_dist  
0                   57155      0.001112  
1              

2766689

In [165]:
# Filter out remaining data coordinates that are too close together
# Count how many points share a closest_location because it is likely they will be near each other
closest_locations_df['Count'] = closest_locations_df.groupby(['user_id', 'closest_location'])['closest_location'].transform('count')

# remove rows with a location count > 1 but keep the first occurance
ones = closest_locations_df.loc[closest_locations_df['Count'] == 1]
twos = closest_locations_df.loc[closest_locations_df['Count'] > 1]

# remove duplicate nearest distances
twos.drop_duplicates(subset=['user_id', 'closest_location', 'Count'], keep='first', inplace=True)

# merge back one of the duplicate rows
concat_df = pd.concat([ones,twos])
concat_df

# Saved another 629593 rows! (2766689 - 2137096)

/var/folders/th/h7c9tz61505fhg7ds00qjhlm0000gn/T/ipykernel_873/2249486294.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twos.drop_duplicates(subset=['user_id', 'closest_location', 'Count'], keep='first', inplace=True)


,user_id,location_id,latitude,longitude,trendsetter_score,closest_location,closest_dist,Count
2,0,9382,30.263886,-97.737980,9506,211286,0.003099,1
4,0,15245,30.264298,-97.732980,9506,9382,0.005017,1
5,0,9410,30.201557,-97.667127,9506,123535,0.084309,1
8,0,260957,40.729768,-73.998535,9506,16397,0.001467,1
9,0,15372,30.261599,-97.758581,9506,16516,0.002068,1
...,...,...,...,...,...,...,...,...
2766662,196577,394036,51.748408,-0.457842,28,496276,0.030357,2
2766666,196577,3430660,51.880535,-0.625102,28,845387,0.051311,2
2766667,196577,48368,51.441187,0.366846,28,47207,0.024960,2
2766678,196578,964995,51.745652,-0.487572,17,797460,0.000735,2


In [167]:
# Count how many times a user appears in our dataset
concat_df['total_count'] = concat_df.groupby(['user_id'])['user_id'].transform('count')

# Drop Unnecessary Columns
concat_df.drop(columns=['location_id', 'closest_location', 'closest_dist', 'Count'], axis=1, inplace=True)
concat_df

,user_id,latitude,longitude,trendsetter_score,total_count
2,0,30.263886,-97.737980,9506,80
4,0,30.264298,-97.732980,9506,80
5,0,30.201557,-97.667127,9506,80
8,0,40.729768,-73.998535,9506,80
9,0,30.261599,-97.758581,9506,80
...,...,...,...,...,...
2766662,196577,51.748408,-0.457842,28,9
2766666,196577,51.880535,-0.625102,28,9
2766667,196577,51.441187,0.366846,28,9
2766678,196578,51.745652,-0.487572,17,13


Problem 1: The Convex Hull method will not work with duplicate points.

Problem 2: The Convex Hull method will not work with points that are too close to each other either. 

Problem 3: The Convex Hull method needs at least 3 points to work. Therefore, I will make a dataframe for users with only 2 coordinate points and calculate distance through brute force. Then, I will make a dataframe for users with only 3 or more coordinate points and calculate the hull distance. Lastly, merge and compare them to create the Top 10.

In [183]:
# Make a dataframe for users with 3+ locations visited (used for convex_hull method)
convex_hull_df = concat_df.loc[concat_df['total_count'] >= 3]

# Make a dataframe for users with < 3 locations (used for brute force method)
brute_force_df = concat_df.loc[(concat_df['total_count'] < 3) & (concat_df['total_count'] > 1)]

# Drop Unnecessary Columns
convex_hull_df.drop(columns=['total_count'], axis=1, inplace=True)
brute_force_df.drop(columns=['total_count'], axis=1, inplace=True)

# Sort dataframes by user_id so all of their coordinates are together and to reduce complexity
convex_hull_df = convex_hull_df.sort_values(by='user_id')
brute_force_df = brute_force_df.sort_values(by='user_id')


/var/folders/th/h7c9tz61505fhg7ds00qjhlm0000gn/T/ipykernel_873/3412628254.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  convex_hull_df.drop(columns=['total_count'], axis=1, inplace=True)
/var/folders/th/h7c9tz61505fhg7ds00qjhlm0000gn/T/ipykernel_873/3412628254.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brute_force_df.drop(columns=['total_count'], axis=1, inplace=True)


### Calculate the Largest Distance a User Has Traveled Based on All of the Coordinate Pairs They Have Been To (Brute Force Approach)

In [209]:
# Time complexity: O(n^2)
# Note: Haversine Distance Formula Was Defined During the Subset Data Process
import numpy as np
from itertools import combinations

# Initialize dictionary to store the largest distance for each user
largest_distances = {}

# Group data by distinct user_id to iterate through each user
distance_data = brute_force_df.groupby('user_id')

# Iterate through each user and calculate their largest distance (user_group = all the rows belonging to current user)
for user, user_group in distance_data:
    distances = [] # list to store all of a user's distances

    # Create combinations of every user's latitude & longitude pairs at each user index
    # Then, calculate the distance between all pairs of coordinates for a user
    for pair in combinations(user_group.index, 2):
        lat1, lon1 = user_group.loc[pair[0], ['latitude', 'longitude']]
        lat2, lon2 = user_group.loc[pair[1], ['latitude', 'longitude']]
        distances.append(get_radius_from_lat_lon_in_km(lat1, lon1, lat2, lon2)) # add each radius to list

    # Find the largest radius for this user
    largest_distance = max(distances)
    largest_distances[user] = largest_distance # keep track of the largest radius for each user in dictionary
 
# Note: 13,000+ rows run in 10 seconds :D

In [193]:
# Convert the Brute Force dictionary to a DataFrame
brute_largest_radius_df = pd.DataFrame(list(largest_distances.items()), columns=['user_id', 'largest_radius'])

brute_largest_radius_df

,user_id,largest_radius
0,61,7.916146
1,80,0.551031
2,123,1975.594780
3,144,34.459614
4,229,0.812033
...,...,...
6501,196466,0.666451
6502,196479,5.289240
6503,196487,13.762559
6504,196520,0.855332


### Calculate the Largest Distance a User Has Traveled With the Convex Hull Method

In [196]:
# Complexity: O(nlogn)
# Note: Haversine Distance Formula Was Defined During the Subset Data Process
import numpy as np
from itertools import combinations
from scipy.spatial import ConvexHull

# Initialize dictionary to store the largest distance for each user
largest_distances = {}

# Group data by distinct user_id to iterate through each user
distance_data = convex_hull_df.groupby('user_id') 

# Iterate through each user_id and calculate their largest distance
for user, user_group in distance_data:
    coordinates = user_group[['latitude', 'longitude']].values # Store all lat & lon pairs for the user_id in an array

    # Create a Convex Hull object from all of a user's coordinates
    hull = ConvexHull(coordinates)

    # Get vertices of convex hull
    hull_vertices = coordinates[hull.vertices]

    # Calculate distances between vertices
    distances = []

    # Iterate through all verticies of the Convex Hull
    for i in range(len(hull_vertices)):
        for j in range(i + 1, len(hull_vertices)): # Avoid calculating distance between the same pair of vertices twice 
            lat1, lon1 = hull_vertices[i]
            lat2, lon2 = hull_vertices[j]
            distance = get_radius_from_lat_lon_in_km(lat1, lon1, lat2, lon2) # calculate radius
            distances.append(distance) # store list of user's radiuses

    # Find the largest radius for this user
    largest_distance = max(distances)
    largest_distances[user] = largest_distance # add largest distance to dictionary

    # Note: 2 million+ rows run in under 2 minutes :D

### Create Final Tables

In [206]:
# Convert the Convex Hull dictionary to a DataFrame
# Add column displaying the largest distance for each user
convex_largest_radius_df = pd.DataFrame(list(largest_distances.items()), columns=['user_id', 'largest_radius'])

# Merge Convex Hull and Brute Force dataframes
final_df = convex_largest_radius_df.merge(brute_largest_radius_df, on='user_id', how='left')

# Add trendsetters score column
final_df = final_df.merge(filtered_df, on='user_id', how='left')

# Remove duplicate user_id rows
final_df = final_df.drop_duplicates(subset=['user_id'])

# Drop Unnecessary Columns
final_df.drop(columns=['largest_radius_y', 'latitude', 'longitude', 'location_id'], axis=1, inplace=True)

# Rename 'largest_radius' column
final_df = final_df.rename(columns={'largest_radius_x':'largest_radius'})

# Sort by trendsetter_score & radius to create final tables
final_score_df = final_df.sort_values(by=['trendsetter_score'], ascending=False)
final_radius_df = final_df.sort_values(by=['largest_radius'], ascending=False)

## Deliverables 
### Trendsetters Ranked By Score (Highest to Lowest)

In [207]:
final_score_df = final_score_df.head(10)
final_score_df

,user_id,largest_radius,trendsetter_score
2115799,48860,7797.931067,60714
3132,22,2178.414601,58371
47618,267,1389.628837,49310
70896,401,672.844768,47750
114170,588,225.685489,45694
21584,138,1961.124544,43065
53024,312,4837.165556,41594
61976,350,1465.908090,41324
89873,486,1967.025125,41144
18563,130,1748.147806,40486


### Trendsetters Ranked By Radius of Influence (Highest to Lowest)

In [208]:
final_radius_df = final_radius_df.head(10)
final_radius_df

,user_id,largest_radius,trendsetter_score
1561312,27599,9933.112362,757
40974,234,9779.867269,7726
116552,599,9644.412484,17938
2787923,100826,9607.088441,7198
479283,3237,9607.038860,4591
3065336,111224,9606.209756,2083
588911,4362,9587.796542,15371
365059,2350,9586.779903,16298
2586851,83799,9533.084126,131
3712944,159859,9518.323180,1822
